**Proyecto 3 Agrupación de datos sobre obesidad**

Trabajo realizado por:

* Diego Benavides

**Resumen**
Base de datos a Estudiar: https://archive.ics.uci.edu/dataset/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition 

contiene información sobre niveles de obesidad en personas de tres países de latinoamérica: Colombia, México y Perú de acuerdo a su estado físico y sus hábitos alimenticios.

La base de datos contiene 17 atributos y 2111 registros, con la etiqueta NObesity la cual denota el nivel de obesidad, esto permite clasificar los datos en los valores de Peso insuficiente, Peso normal, Sobrepeso nivel I, Sobrepeso nivel II, Obesidad tipo I, Obesidad tipo II and obesidad tipo III. 
Del total de los datos, 77% fueron generados sintéticamente y el restante fue obtenido mediente recopilación directa con individuos a través de un portal web.




**Importar librerías**

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score

**Cargar Base de datos y mostrar información**

Hacer un resumen de la información del dataframe y detallar el tipo de variables.

In [6]:
# Cargar datos:
df = pd.read_csv('../Datos/ObesityDataSet_raw_and_data_sinthetic.csv')

In [12]:
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [20]:
# Dimensión del dataframe
df.shape

(2111, 17)

In [10]:
# Información de Dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [22]:
# Identificar valores nulos
display(df.isnull().sum())

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [24]:
#Valores de la etiqueta NObeyesdad
df['NObeyesdad'].value_counts()

NObeyesdad
Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: count, dtype: int64

In [28]:
# Contar variables numéricas
num_variables = df.select_dtypes(include=['number']).shape[1]

# Contar variables cualitativas
non_num_variables = df.select_dtypes(exclude=['number']).shape[1]

print(f'Número de variables numéricas: {num_variables}')
print(f'Número de variables no numéricas: {non_num_variables}')

Número de variables numéricas: 8
Número de variables no numéricas: 9


**Resumen de la información de los datos**

- El dataframe consta de 17 columnas 2111 registros.
- De las 17 variables, 8 son numéricas (float64) y  son cualitativas (object).
- El dataframe no contiene valores nulos.
- Se individualizan los valores por resultadod e la etiqueta NObeyesdad para tener una idea de su distribución por sus distintos valores en el dataframe.


**Exploración de los datos**

In [34]:
df.describe()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000
